In [5]:
import csv
import gzip
import time
import os
import pandas as pd
import re
from numpy import where as npwhere
from functools import reduce

def mapMethytilatedCitosine(metfile,intersection_file_path,output_file_path):
    chrs_data = {}
    with gzip.open(metfile, 'rt') as f:
        for line in f:
            chr_index = line.split('\t')[0]
            coordinate_index = int(line.split('\t')[1])
            try:
                chrs_data[chr_index][coordinate_index] = line[:-5]
            except:
                chrs_data[chr_index] = {coordinate_index:line[:-5]}

    with open(intersection_file_path) as f:
        reader = csv.reader(f, delimiter='\t')
        data = list(reader)

    with open(output_file_path, 'w') as whateverBox:
        for each_box in data:
            box_areas = {}
            box_start = int(each_box[1])
            box_end = int(each_box[2])
            chr_location = each_box[0]
            sequence_identifier = each_box[3]
            # tenemos que aplicar rango desde una posicion inicial porque es un archivo 0 based, si fuese 1 based el rango estaría mal y tendria que restarle
            # uno al archivo al box start
            box_areas['box'] = list(range(box_start, box_end))

            for boxarea in box_areas:
                for coordinate in box_areas[boxarea]:
                    matchline = chrs_data[chr_location].get(coordinate, False)
                    if matchline:
                        whateverBox.write('{}\t{}\t{}\n'.format(matchline, sequence_identifier, boxarea))
                        

def loadFileRemoveMtsAndLowMetReport(replicate):
        bxdf = pd.read_csv(
            replicate,
            sep='\t', 
            names=['chr', 'position', 'strand', 'mets', 'nomets', 'context', 'boxID', 'side'],
            usecols=['chr', 'position', 'strand', 'boxID','mets', 'nomets', 'context']
                )
        bxdf.drop(bxdf[(bxdf.chr == 'Mt') | (bxdf.chr == 'Pt')].index, inplace=True)
        bxdf = bxdf[bxdf['mets'] + bxdf['nomets'] >= 4]
        return bxdf

def calculateValueAsPercentageOfMetCs(replicatedf,replicanamedf):
    replicatedf[replicanamedf] = replicatedf['mets'] / (replicatedf['nomets'] + replicatedf['mets'])
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf

In [6]:
working_folder = '/home/joaquin/projects/methylation/data/bisulfite_rep1_rep2_rep3/finalreps'
intersection_file = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'
basedmrfolder = '/home/joaquin/projects/methylation/methylation_jose/met_site'
experimentsClasification = {}
for root, dirs, files in os.walk(working_folder):

    for file in files:
        if file.endswith('CX_report.txt.gz'):
            replicate, hour, condition =root.strip().split('/')[-3:]
            experimentName = hour+condition
            experimentsClasification.setdefault(experimentName, [])
            targetMetFilename = root+'/'+file
            finalnamedestination = basedmrfolder+'/'+experimentName+replicate+'_CX_report.txt.gz'
            outputfile = targetMetFilename[:-3] + 'allThePossiblePeaksnine.tsv'
            experimentsClasification[experimentName].append(outputfile)
            print(outputfile)
            mapMethytilatedCitosine(targetMetFilename,intersection_file,outputfile)

            

/home/joaquin/projects/methylation/data/bisulfite_rep1_rep2_rep3/finalreps/rep3/6/ACC/WGBS8_FDLM220151345-1a_H5HGMDSX3_L1_1_val_1_bismark_bt2_pe.multiple.deduplicated.CX_report.txtallThePossiblePeaksnine.tsv
/home/joaquin/projects/methylation/data/bisulfite_rep1_rep2_rep3/finalreps/rep3/6/Mock/WGBS2_FDLM220151339-1a_H5HGMDSX3_L1_1_val_1_bismark_bt2_pe.multiple.deduplicated.CX_report.txtallThePossiblePeaksnine.tsv
/home/joaquin/projects/methylation/data/bisulfite_rep1_rep2_rep3/finalreps/rep3/6/JA/WGBS5_FDLM220151342-1a_H5HGMDSX3_L1_1_val_1_bismark_bt2_pe.multiple.deduplicated.CX_report.txtallThePossiblePeaksnine.tsv
/home/joaquin/projects/methylation/data/bisulfite_rep1_rep2_rep3/finalreps/rep3/24/ACC/WGBS9_FDLM220151346-1a_H5GTJDSX3_L1_1_val_1_bismark_bt2_pe.multiple.deduplicated.CX_report.txtallThePossiblePeaksnine.tsv
/home/joaquin/projects/methylation/data/bisulfite_rep1_rep2_rep3/finalreps/rep3/24/Mock/WGBS3_FDLM220151340-1a_H5HGMDSX3_L1_1_val_1_bismark_bt2_pe.multiple.deduplicate

In [7]:
allthedata = []
for experimentCondition in experimentsClasification:
    listOfDfReplicatesPercentageOfMetCs = []
    listOfDfReplicatesCualitative = []
    listOfNamesReplicates = []
    for replicatePath in experimentsClasification[experimentCondition]:

        replicaname =  experimentCondition+replicatePath.split('/')[-4]
        replicateDf = loadFileRemoveMtsAndLowMetReport(replicatePath)

        replicateDfPercentageOfMetCs = calculateValueAsPercentageOfMetCs(replicateDf,replicaname)

#         replicateDfCualitative = calculateValueAsCualitative(replicateDf,replicaname)
#         print(replicateDfCualitative)
        listOfNamesReplicates.append(replicaname)
        listOfDfReplicatesPercentageOfMetCs.append(replicateDfPercentageOfMetCs)
#         listOfDfReplicatesCualitative.append(replicateDfCualitative)
        
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['chr','position', 'strand','context','boxID'],
                                                how='outer'), listOfDfReplicatesPercentageOfMetCs)
    print(df_merged)
    df_merged = df_merged.dropna()
    df_merged[experimentCondition] = df_merged[listOfNamesReplicates].mean(axis=1)
    print(df_merged)
    df_merged = df_merged.drop(columns=listOfNamesReplicates)
    allthedata.append(df_merged)
finaldf = reduce(lambda  left,right: pd.merge(left,right,on=['chr','position', 'strand','context','boxID'],
                                                how='outer'), allthedata)
finaldf = finaldf.dropna()

         chr  position strand context       boxID  6ACCrep3  6ACCrep1  \
0          1      1438      +      CG      1:1538       0.0       0.0   
1          1      1439      -      CG      1:1538       0.0       NaN   
2          1      1444      -     CHH      1:1538       0.0       NaN   
3          1      1449      -     CHH      1:1538       0.0       NaN   
4          1      1450      -     CHH      1:1538       0.0       NaN   
...      ...       ...    ...     ...         ...       ...       ...   
4216988    5  26942835      +      CG  5:26942928       NaN       NaN   
4216989    5  26942844      +     CHH  5:26942928       NaN       NaN   
4216990    5  26942845      +     CHG  5:26942928       NaN       NaN   
4216991    5  26942846      +      CG  5:26942928       NaN       NaN   
4216992    5  26974913      +     CHH  5:26974854       NaN       NaN   

         6ACCrep2  
0             0.0  
1             0.0  
2             0.0  
3             0.0  
4             0.0  
...

         chr  position strand context       boxID  24ACCrep3  24ACCrep1  \
0          1      1438      +      CG      1:1538   0.000000   0.000000   
1          1      1439      -      CG      1:1538   0.000000   0.000000   
2          1      1444      -     CHH      1:1538   0.000000   0.000000   
3          1      1449      -     CHH      1:1538   0.000000   0.000000   
4          1      1450      -     CHH      1:1538   0.000000   0.000000   
...      ...       ...    ...     ...         ...        ...        ...   
4114188    5  26974940      -     CHH  5:26974854   0.222222   0.214286   
4114189    5  26974941      -     CHH  5:26974854   0.000000   0.000000   
4114190    5  26974943      +     CHH  5:26974854   0.200000   0.000000   
4114191    5  26974945      +     CHH  5:26974854   0.166667   0.000000   
4114192    5  26974952      +     CHH  5:26974854   0.000000   0.000000   

         24ACCrep2     24ACC  
0         0.000000  0.000000  
1         0.000000  0.000000  
2     

         chr  position strand context       boxID  1Mockrep3  1Mockrep1  \
0          1      1438      +      CG      1:1538        0.0        0.0   
1          1      1439      -      CG      1:1538        0.0        0.0   
2          1      1444      -     CHH      1:1538        0.0        NaN   
3          1      1449      -     CHH      1:1538        0.0        NaN   
4          1      1450      -     CHH      1:1538        0.0        NaN   
...      ...       ...    ...     ...         ...        ...        ...   
4227597    5  26965672      -     CHH  5:26965576        NaN        NaN   
4227598    5  26965674      -     CHH  5:26965576        NaN        NaN   
4227599    5  26967680      +     CHH  5:26967769        NaN        NaN   
4227600    5  26967681      +     CHH  5:26967769        NaN        NaN   
4227601    5  26967683      +     CHH  5:26967769        NaN        NaN   

         1Mockrep2  
0             0.00  
1             0.00  
2             0.00  
3             0

In [22]:
import pandas as pd 
peaksfile = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'

peaks = pd.read_csv(peaksfile, sep='\t', names=['chrm','start','end','boxID'])


df  = finaldf[(finaldf['context'] == 'CG')]
metadata = df.iloc[:,4:5]
values = df.iloc[:,5:]

merged = metadata.merge(values, left_index=True, right_index=True, how='inner')
meandf = merged.groupby('boxID', as_index=False).mean()
peaks = peaks.merge(meandf, on='boxID', how='outer')
peaks = peaks[['chrm','start','end','boxID','1Mock','1JA','1ACC','6Mock','6JA','6ACC','24Mock','24JA','24ACC']]



peaks.to_csv('/home/joaquin/projects/methylation/methylation_jose/met_site/CG_BSMet_mean_allpeaks_no_NaN_CX.tsv',sep='\t', index=False)

# peaks.to_csv('CHG_BSMet_mean.tsv',sep='\t', index=False)

# peaks.to_csv('/home/joaquin/projects/methylation/methylation_jose/met_site/non-CG_BSMet_meam_allpeaks_no_NaN_CX.tsv',sep='\t', index=False)
# meandf['chr'], meandf['start'], meandf['end'] =  zip(*meandf['boxID'].map(lambda x: getpositions(x)))